In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import kurtosis
import requests
%matplotlib inline

In [2]:
df=pd.read_excel('C:\\Users\\nafissaad\\Downloads\\NEDL_PMPT.xlsx',skiprows=[0],index_col='Unnamed: 0')[['JPM','WMT','MSFT','KO','PFE','CAT','SP 500']]
#df.to_excel('C:\\Users\\nafissaad\\Downloads\\NEDL_PMPT.xlsx')
df

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
2011-11-14,24.705299,46.650124,21.537199,24.791821,13.043631,72.899940,103.301254
2011-11-15,24.819157,45.517323,21.683163,24.868612,13.096357,73.307755,103.811737
2011-11-16,23.885593,44.899456,21.139864,24.506557,12.898625,72.310883,102.164955
2011-11-17,23.141773,44.939053,20.710085,24.363930,12.832716,70.845764,100.542900
2011-11-18,23.240440,45.335125,20.515478,24.645535,12.872263,70.936417,100.435875
...,...,...,...,...,...,...,...
2021-11-08,169.059998,149.270004,336.989990,56.330002,48.330002,214.250000,468.929993
2021-11-09,167.779999,149.789993,335.950012,56.490002,47.299999,209.809998,467.380005
2021-11-10,167.619995,148.440002,330.799988,56.720001,49.020000,206.669998,463.619995
2021-11-11,167.610001,148.500000,332.429993,56.740002,50.180000,208.529999,463.769989


In [3]:
returns=df.pct_change()
returns

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
2011-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-11-15,0.004609,-0.024283,0.006777,0.003097,0.004042,0.005594,0.004942
2011-11-16,-0.037615,-0.013574,-0.025056,-0.014559,-0.015098,-0.013598,-0.015863
2011-11-17,-0.031141,0.000882,-0.020330,-0.005820,-0.005110,-0.020261,-0.015877
2011-11-18,0.004264,0.008814,-0.009397,0.011558,0.003082,0.001280,-0.001064
...,...,...,...,...,...,...,...
2021-11-08,0.006010,-0.006390,0.002767,-0.008973,-0.005760,0.040655,0.000854
2021-11-09,-0.007571,0.003484,-0.003086,0.002840,-0.021312,-0.020723,-0.003305
2021-11-10,-0.000954,-0.009013,-0.015330,0.004072,0.036364,-0.014966,-0.008045
2021-11-11,-0.000060,0.000404,0.004927,0.000353,0.023664,0.009000,0.000324


In [4]:
matrix=pd.DataFrame(columns=[returns.columns],index=['Return,historical','Beta','Downside beta','Returns,CAPM','Returns,D-CAPM','Weights'])
matrix

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
"Return,historical",NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Downside beta,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Returns,CAPM",NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Returns,D-CAPM",NaN,NaN,NaN,NaN,NaN,NaN,NaN
Weights,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
risk_free_ratre_annual=0.0157

In [6]:
from scipy import stats
stats.linregress(returns['SP 500'].dropna(),returns['JPM'].dropna())[0]

1.2576308464675616

In [7]:
matrix.loc['Beta','JPM'][0]

nan

In [8]:
matrix.loc['Return,historical']=[(df[i][-1]/df[i][1])**(1/10)-1 for i in returns.columns]
matrix.loc['Beta']=[ stats.linregress(returns['SP 500'].dropna(),returns[i].dropna())[0] for i in returns.columns]
matrix.loc['Returns,CAPM']=[risk_free_ratre_annual+matrix.loc['Beta',i][0]*(matrix.loc['Return,historical','SP 500'][0]-risk_free_ratre_annual) for i in returns.columns]
matrix.loc['Downside beta']=[stats.linregress(returns[returns['SP 500']<0]['SP 500'],returns[returns['SP 500']<0][i])[0] for i in returns.columns]
matrix.loc['Returns,D-CAPM']=[risk_free_ratre_annual+matrix.loc['Downside beta',i][0]*(matrix.loc['Return,historical','SP 500'][0]-risk_free_ratre_annual) for i in returns.columns]
matrix.loc['Weights'][:-1]=1/6


In [9]:
matrix

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500
"Return,historical",0.20992,0.124962,0.315572,0.085736,0.142738,0.110595,0.162337
Beta,1.257631,0.549411,1.171371,0.671961,0.722679,1.167853,1.0
Downside beta,1.23808,0.504701,1.126768,0.747261,0.705188,1.040177,1.0
"Returns,CAPM",0.200115,0.096264,0.187467,0.114234,0.121672,0.186951,0.162337
"Returns,D-CAPM",0.197249,0.089708,0.180926,0.125276,0.119107,0.168229,0.162337
Weights,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,NaN


In [10]:
returns['Portfolio']=matrix.loc['Weights'][:-1].values.dot(returns[returns.columns[:-1]].fillna(0).values.T)
returns

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500,Portfolio
2011-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2011-11-15,0.004609,-0.024283,0.006777,0.003097,0.004042,0.005594,0.004942,-0.000027
2011-11-16,-0.037615,-0.013574,-0.025056,-0.014559,-0.015098,-0.013598,-0.015863,-0.019917
2011-11-17,-0.031141,0.000882,-0.020330,-0.005820,-0.005110,-0.020261,-0.015877,-0.01363
2011-11-18,0.004264,0.008814,-0.009397,0.011558,0.003082,0.001280,-0.001064,0.003267
...,...,...,...,...,...,...,...,...
2021-11-08,0.006010,-0.006390,0.002767,-0.008973,-0.005760,0.040655,0.000854,0.004718
2021-11-09,-0.007571,0.003484,-0.003086,0.002840,-0.021312,-0.020723,-0.003305,-0.007728
2021-11-10,-0.000954,-0.009013,-0.015330,0.004072,0.036364,-0.014966,-0.008045,0.000029
2021-11-11,-0.000060,0.000404,0.004927,0.000353,0.023664,0.009000,0.000324,0.006381


In [11]:
Annual_returns=matrix.loc['Weights'][:-1].dot(matrix.loc['Returns,CAPM'][df.columns[:-1].values])
Annual_returns

0.15111714195707227

In [12]:
Historical_return_daily=returns['Portfolio'].mean()
Historical_return_daily

0.0007057748682600625

In [13]:
Volatility=returns['Portfolio'].std()*np.sqrt(252)
Volatility

0.16334873482385878

In [14]:
Annual_target=0.1
daily_target=(1+Annual_target)**(1/252)-1
daily_target

0.0003782865315342665

In [15]:
returns['Under performed our daily Target']=[i if i>0 else 0 for i in (daily_target-returns['Portfolio'])]
returns

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500,Portfolio,Under performed our daily Target
2011-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000378
2011-11-15,0.004609,-0.024283,0.006777,0.003097,0.004042,0.005594,0.004942,-0.000027,0.000405
2011-11-16,-0.037615,-0.013574,-0.025056,-0.014559,-0.015098,-0.013598,-0.015863,-0.019917,0.020295
2011-11-17,-0.031141,0.000882,-0.020330,-0.005820,-0.005110,-0.020261,-0.015877,-0.01363,0.014008
2011-11-18,0.004264,0.008814,-0.009397,0.011558,0.003082,0.001280,-0.001064,0.003267,0.000000
...,...,...,...,...,...,...,...,...,...
2021-11-08,0.006010,-0.006390,0.002767,-0.008973,-0.005760,0.040655,0.000854,0.004718,0.000000
2021-11-09,-0.007571,0.003484,-0.003086,0.002840,-0.021312,-0.020723,-0.003305,-0.007728,0.008106
2021-11-10,-0.000954,-0.009013,-0.015330,0.004072,0.036364,-0.014966,-0.008045,0.000029,0.000349
2021-11-11,-0.000060,0.000404,0.004927,0.000353,0.023664,0.009000,0.000324,0.006381,0.000000


In [16]:
returns['Upside']=[i if i>0 else 0 for i in (returns['Portfolio']-Historical_return_daily)]
returns['Downside']=[i if i>0 else 0 for i in (Historical_return_daily-returns['Portfolio'])]
returns

,JPM,WMT,MSFT,KO,PFE,CAT,SP 500,Portfolio,Under performed our daily Target,Upside,Downside
2011-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000378,0.000000,0.000706
2011-11-15,0.004609,-0.024283,0.006777,0.003097,0.004042,0.005594,0.004942,-0.000027,0.000405,0.000000,0.000733
2011-11-16,-0.037615,-0.013574,-0.025056,-0.014559,-0.015098,-0.013598,-0.015863,-0.019917,0.020295,0.000000,0.020623
2011-11-17,-0.031141,0.000882,-0.020330,-0.005820,-0.005110,-0.020261,-0.015877,-0.01363,0.014008,0.000000,0.014336
2011-11-18,0.004264,0.008814,-0.009397,0.011558,0.003082,0.001280,-0.001064,0.003267,0.000000,0.002561,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-08,0.006010,-0.006390,0.002767,-0.008973,-0.005760,0.040655,0.000854,0.004718,0.000000,0.004012,0.000000
2021-11-09,-0.007571,0.003484,-0.003086,0.002840,-0.021312,-0.020723,-0.003305,-0.007728,0.008106,0.000000,0.008434
2021-11-10,-0.000954,-0.009013,-0.015330,0.004072,0.036364,-0.014966,-0.008045,0.000029,0.000349,0.000000,0.000677
2021-11-11,-0.000060,0.000404,0.004927,0.000353,0.023664,0.009000,0.000324,0.006381,0.000000,0.005676,0.000000


In [17]:
Downside_vol=returns['Downside'].std()*np.sqrt(252)
Downside_vol

0.10362444741905431

In [18]:
Upside_var=sum(returns['Upside']**2)/len(returns['Upside'])
downside_var=sum(returns['Downside']**2)/len(returns['Downside'])
Upside_var,downside_var

(5.1683445141087924e-05, 5.415865062056657e-05)

In [19]:
Upside_var+downside_var

0.0001058420957616545

In [20]:
Upside_var_pct=Upside_var/(Upside_var+downside_var)
Downside_var_pct=downside_var/(Upside_var+downside_var)
Downside_var_pct,Upside_var_pct

(0.5116929160447302, 0.4883070839552697)

In [21]:
volatility_skewness=Upside_var_pct/Downside_var_pct
volatility_skewness#1 means equal side of risk less than 1 means more down side risk

0.9542971353400246

In [22]:
Sharp_ratio=(Annual_returns-risk_free_ratre_annual)/Volatility
Sharp_ratio

0.8290063715711937

In [23]:
Sortino_ratio=(Annual_returns-Annual_target)/Downside_vol
Sortino_ratio

0.4932922995512439